Jupyter notebook för att göra kluster för Livsmedelsverkets databas

In [56]:
import sqlite3
import numpy as np
from sklearn.cluster import KMeans
import plotly
import plotly.graph_objs as go
import plotly.plotly as py

In [57]:
# Variable to toggle printouts
debug = False

Vi har konverterat en nedladdad Excel-fil till csv och sen till Sqlite 3. 

Den innehåller alla Livsmedelsverkets 59 näringsämnen. Här jobbar vi bara med kolhydrater, fett och protein.

Först läser vi in från databasen och lägger värdena i arrayen *result*

In [58]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
columnObject = []

        #Kolla om klusterkolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
#print (columnObject)
columns = np.array(columnObject).tolist()

conn.close()

In [59]:
for column in columns:
    print (column[0],column[1])

0 Livsmedelsnamn
1 Livsmedelsnummer
2 Energi_kcal
3 Energi_kJ
4 Kolhydrater_g
5 Fett_g
6 Protein_g
7 Fibrer_g
8 Vatten_g
9 Alkohol_g
10 Aska_g
11 Monosackarider_g
12 Disackarider_g
13 Sackaros_g
14 Fullkorn_totalt_g
15 Sockerarter_g
16 Summa_mattade_fettsyror_g
17 Fettsyra_4_0_10_0_g
18 Fettsyra_12_0_g
19 Fettsyra_14_0_g
20 Fettsyra_16_0_g
21 Fettsyra_18_0_g
22 Fettsyra_20_0_g
23 Summa_enkelomattade_fettsyror_g
24 Fettsyra_16_1_g
25 Fettsyra_18_1_g
26 Summa_fleromattade_fettsyror_g
27 Fettsyra_18_2_g
28 Fettsyra_18_3_g
29 Fettsyra_20_4_g
30 EPA_Fettsyra_20_5_g
31 DPA_Fettsyra_22_5_g
32 DHA_Fettsyra_22_6_g
33 Kolesterol_mg
34 Retinol_mikrog
35 Vitamin_A_mikrog
36 beta_Karoten_mikrog
37 Vitamin_D_mikrog
38 Vitamin_E_mg
39 Vitamin_K_mikrog
40 Tiamin_mg
41 Riboflavin_mg
42 Vitamin_C_mg
43 Niacin_mg
44 Niacinekvivalenter_mg
45 Vitamin_B6_mg
46 Vitamin_B12_mikrog
47 Folat_mikrog
48 Fosfor_mg
49 Jod_mikrog
50 Jarn_mg
51 Kalcium_mg
52 Kalium_mg
53 Magnesium_mg
54 Natrium_mg
55 Salt_g
56 Selen_

In [60]:
# Välj vilka näringsvärden som ska läsas ur databasen. Gör en lista av dem i valueList
valueList = [1,5,35,53]
columnsToQuery=[]
for index, item in enumerate(valueList):
    columnsToQuery.append(columns[item][1])

# Gör en sträng av listan av kolumner
columnsToQuery = ', '.join(columnsToQuery)
if debug:
    print (columnsToQuery)

In [61]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
result = []
rows = curs.execute('select '+columnsToQuery+' from livs')
for row in rows:
        result.append(row)

rows = np.array(result)
#print (rows)

conn.close()

Livsmedelsnumret ska inte vara med i klustringen så vi gör två matriser. 

allFactors innehåller Livsmedelsnumret, och dataSet allt utom Livsmedelsnumret

In [62]:
allFactors = np.array(result)
dataSet = allFactors[:,1:]

För klustringsalgoritmerna behöver vi använda numpy och göra en np-array

För fem livsmedel, kolumner för kolhydrater, fett och protein

<pre>[[   0.    71.     7. ]
 [   0.    85.     2.8]
 [   0.   100.     0. ]
 [   0.   100.     0. ]
 [   0.5   80.     0.5]]</pre>



In [63]:
if debug:
    print (dataSet)

Funktion för att göra själva klustringen enligt k-means.

K-means förklaras relativt enkelt här: http://bigdata-madesimple.com/possibly-the-simplest-way-to-explain-k-means-algorithm/

In [64]:
def cluster(dataSet):

    kmeans = KMeans(n_clusters=numClusters)
    kmeans.fit(dataSet)

    centroids = kmeans.cluster_centers_
    assignedCluster = kmeans.labels_

    if debug:
        print ("Centroids \n", centroids)
        print
        print (assignedCluster)

    return assignedCluster

In [66]:
print(dataSet)

[['71.0' '22.0' '3.0']
 ['85.0' '4.0' '1.0']
 ['100.0' '9.0' '1.0']
 ..., 
 ['0.5' '0.0' '4.0']
 ['3.55' '121.0' '84.5']
 ['4.8' '0.0' '22.0']]


När man kör klustringsalgoritmen returneras en array av vilket kluster varje livsmedel tillhör. 

Man kan även få ut mittpunkterna i varje kluster genom variabeln *centroids*

Ett exempel: 

<pre>
centroids:
[[   0.    100.      0.  ]
 [   0.25   82.5     1.65]
 [   0.     71.      7.  ]]

assignedCluster:
[2 1 0 0 1]</pre>

In [67]:
numClusters = 5
assignedCluster = cluster(dataSet)

ValueError: could not convert string to float: 

In [68]:
assignedCluster = np.array([assignedCluster])
print(assignedCluster)


[[[[1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 2 1 1 1]]]]


In [69]:
new=np.concatenate((allFactors[:,:1], assignedCluster.T), axis=1)
print (new)

ValueError: all the input arrays must have same number of dimensions

I princip kan det här göras med många fler näringsvärden där varje näringsvärde blir en "dimension". 

Eftersom vi vill visa klustringen i ett 3D-diagram har vi begränsat oss till tre dimensioner i exemplet.

## Visualisering med Plotly

I Plotly svarar varje näringsvärde mot en axel.

För att Plotly ska kunna plotta varje axel för sig transponerar vi data så vi får en rad för x-axeln, en för y-axeln och en för z.

Ett exempel:

<pre>
[[   0.     0.     0.     0.     0.5]
 [  71.    85.   100.   100.    80. ]
 [   7.     2.8    0.     0.     0.5]]
</pre>

In [15]:
dataToChart = dataSet.transpose()

In [16]:
if debug:
    print (dataToChart)

[['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '12' '13' '17' '18' '20' '21'
  '22' '23' '24' '25']
 ['71.0' '85.0' '100.0' '100.0' '80.0' '60.0' '70.0' '60.0' '60.0' '82.0'
  '80.0' '80.0' '38.0' '35.0' '40.0' '87.81' '40.21' '42.21' '56.0' '78.81']
 ['22.0' '4.0' '9.0' '0.0' '919.2' '761.6' '800.0' '800.0' '800.0' '693.1'
  '900.0' '800.0' '800.0' '800.0' '789.6' '53.7' '255.8' '58.3' '35.0'
  '59.1']
 ['0.03' '0.03' '0.01' '0.0' '2.0' '1.0' '0.9' '1.2' '0.9' '1.5' '1.5'
  '1.5' '1.25' '0.5' '1.25' '0.56' '1.03' '1.66' '0.69' '0.67']]


För att visualisera vilka kluster som ett livsmedel ligger så ger vi varje kluster olika färger. I Plotly svarar detta mot att varje kluster hamnar i ett separat "trace"

Därför behöver vi gå igenom vårt *dataToChart* och separera ut vilka livsmedel som hör till vilket kluster.

Då behöver vi en tvådimensionell array där storleken på första dimensionen är antal kluster och storleken på andra dimensionen är tre (eftersom vi visualiserar tre dimensioner)

In [17]:
# create multi dimensional array of data by label
segmentedData = [[[] for _ in range(4)] for _ in range(numClusters)]


Det är lite krångligt att visualisera den här strukturen. Så här blir den med fem kluster (numClusters) och tre faktorer för klustringen (fett, protein, kolhydrater) 
<pre>
[
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []], 
    [[], [], []]
]
</pre>

(Om man väljer att klustra med fler än tre faktorer är det svårt att visualisera alla, utan man måste välja bort någon faktor)

In [18]:
jobbigArray = np.array(segmentedData)
print (jobbigArray)

[]


In [258]:
for num, cluster in enumerate(assignedCluster):
        if debug:
            print (str(num) + ' ' + str(cluster))
        segmentedData[cluster][0].append(dataToChart[0][num])
        segmentedData[cluster][1].append(dataToChart[1][num])
        segmentedData[cluster][2].append(dataToChart[2][num])
        segmentedData[cluster][3].append(dataToChart[3][num])
        
jobbigArray = np.array(segmentedData)
print (jobbigArray.ndim)

0 [0 3 1 1 3 2 4 2 2 3]


TypeError: only integer scalar arrays can be converted to a scalar index

In [12]:
if debug:
    print(segmentedData)

[[[13.1, 9.1699999999999999, 10.050000000000001, 2.5, 2.1000000000000001, 9.0999999999999996, 0.90000000000000002, 2.8999999999999999, 3.6000000000000001, 7.2000000000000002, 4.8399999999999999, 10.0, 4.8399999999999999, 5.0300000000000002, 10.9, 4.7000000000000002, 4.8399999999999999, 15.0, 14.300000000000001, 4.7999999999999998, 4.8399999999999999, 4.9800000000000004, 5.2000000000000002, 5.5, 15.0, 14.0, 8.8000000000000007, 14.6, 13.0, 16.800000000000001, 4.8399999999999999, 4.7999999999999998, 4.7999999999999998, 5.1200000000000001, 4.8899999999999997, 10.380000000000001, 10.48, 10.49, 4.6600000000000001, 16.800000000000001, 17.809999999999999, 19.399999999999999, 18.100000000000001, 17.300000000000001, 12.199999999999999, 23.100000000000001, 20.68, 21.850000000000001, 13.9, 9.1999999999999993, 12.5, 7.0999999999999996, 13.9, 8.5999999999999996, 14.15, 9.4900000000000002, 9.1300000000000008, 11.199999999999999, 14.18, 6.5800000000000001, 10.02, 14.52, 14.0, 13.9, 14.99, 20.0, 14.42,

In [13]:
# create traces for plotly
traces = []
baseColor = 100
i = 0
while i < numClusters:
    trace = go.Scatter3d(
        x=segmentedData[i][0],
        y=segmentedData[i][1],
        z=segmentedData[i][3],
        mode='markers',
        marker=dict(
            size=12,
            line=dict(
                color='rgba(baseColor+(i*2), baseColor+(i*2), baseColor+(i*2), 0.14)',
                width=0.5
            ),
            opacity=0.8
        ),
        # @todo: fix names list for plotly
        #text=names
    )
    traces.append(trace)
    i+=1


In [14]:
layout = go.Layout(
        scene=go.Scene(
            xaxis=go.XAxis(title='Carbs', tickprefix='Carbs ', showtickprefix='first'),
            yaxis=go.YAxis(title='Fat', tickprefix='Fat ', showtickprefix='first'),
            zaxis=go.ZAxis(title='Protein', tickprefix='Protein ', showtickprefix='first')
        ),
        height = 800,
        width = 800,
    )

In [15]:
fig = go.Figure(data=traces, layout=layout)    

In [16]:
py.iplot(fig, filename='table1')

In [19]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
columnObject = []

        #Kolla om klusterkolumnen redan finns, annars lägg till den        
for row in conn.execute('PRAGMA table_info (livs);'):
    columnObject.append(row)
columns = np.array(columnObject)
newColumn = 'Kluster'
addColumn = True
for row in columns:
    print (row)
    if newColumn == row[1]:
        addColumn = False
        print (addColumn, newColumn)
if addColumn:
    conn.execute('ALTER TABLE livs ADD COLUMN Kluster REAL;')

#
conn.close()

[0 'Livsmedelsnamn' 'TEXT' 0 None 0]
[1 'Livsmedelsnummer' 'TEXT' 0 None 0]
[2 'Energi_kcal' 'REAL' 0 None 0]
[3 'Energi_kJ' 'REAL' 0 None 0]
[4 'Kolhydrater_g' 'REAL' 0 None 0]
[5 'Fett_g' 'REAL' 0 None 0]
[6 'Protein_g' 'REAL' 0 None 0]
[7 'Fibrer_g' 'REAL' 0 None 0]
[8 'Vatten_g' 'REAL' 0 None 0]
[9 'Alkohol_g' 'REAL' 0 None 0]
[10 'Aska_g' 'REAL' 0 None 0]
[11 'Monosackarider_g' 'REAL' 0 None 0]
[12 'Disackarider_g' 'REAL' 0 None 0]
[13 'Sackaros_g' 'REAL' 0 None 0]
[14 'Fullkorn_totalt_g' 'REAL' 0 None 0]
[15 'Sockerarter_g' 'REAL' 0 None 0]
[16 'Summa_mattade_fettsyror_g' 'REAL' 0 None 0]
[17 'Fettsyra_4_0_10_0_g' 'REAL' 0 None 0]
[18 'Fettsyra_12_0_g' 'REAL' 0 None 0]
[19 'Fettsyra_14_0_g' 'REAL' 0 None 0]
[20 'Fettsyra_16_0_g' 'REAL' 0 None 0]
[21 'Fettsyra_18_0_g' 'REAL' 0 None 0]
[22 'Fettsyra_20_0_g' 'REAL' 0 None 0]
[23 'Summa_enkelomattade_fettsyror_g' 'REAL' 0 None 0]
[24 'Fettsyra_16_1_g' 'REAL' 0 None 0]
[25 'Fettsyra_18_1_g' 'REAL' 0 None 0]
[26 'Summa_fleromattade_fet

## Lägga in kluster i databasen


In [70]:
print(new)

for row in new:
    print (row[0])



[['1' '1']
 ['2' '1']
 ['3' '1']
 ['4' '1']
 ['5' '0']
 ['6' '0']
 ['7' '0']
 ['8' '0']
 ['9' '0']
 ['10' '0']
 ['12' '0']
 ['13' '0']
 ['17' '0']
 ['18' '0']
 ['20' '0']
 ['21' '1']
 ['22' '2']
 ['23' '1']
 ['24' '1']
 ['25' '1']]
1
2
3
4
5
6
7
8
9
10
12
13
17
18
20
21
22
23
24
25


Dags att läsa ner resultatet av klustringen till databasen

In [71]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()


for row in new:
    print(row[1], row[0])
    sql = 'UPDATE livs SET Kluster = '+row[1]+' WHERE Livsmedelsnummer = "'+row[0]+'";'
    print(sql)
    curs.execute(sql)
#curs.execute('UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "2";')
conn.commit()
conn.close()

1 1
UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "1";
1 2
UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "2";
1 3
UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "3";
1 4
UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = "4";
0 5
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "5";
0 6
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "6";
0 7
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "7";
0 8
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "8";
0 9
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "9";
0 10
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "10";
0 12
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "12";
0 13
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "13";
0 17
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "17";
0 18
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "18";
0 20
UPDATE livs SET Kluster = 0 WHERE Livsmedelsnummer = "20";
1 21
UPDATE livs SET Kluster = 1 WHERE Livsmedelsnummer = 

In [72]:
conn = sqlite3.connect('livs.db')  # Create db and establish connection
conn.row_factory = sqlite3.Row
curs = conn.cursor()
   
KlusterObject = []
for row in curs.execute('SELECT Livsmedelsnamn from livs WHERE Kluster = 1;'):
    #print(row)
    KlusterObject.append(row)

        
Kluster = np.array(KlusterObject)
print (Kluster)

conn.close()

[['Talg nöt']
 ['Späck gris']
 ['Ister gris']
 ['Kokosfett']
 ['Majonnäs fett 90%']
 ['Gravlaxsås hovmästarsås']
 ['Gravlaxsås']
 ['Remouladsås']]
